# L7: Build a Crew to Tailor Job Applications

In this lesson, you will built your first multi-agent system.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [5]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool,
  PDFSearchTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./aakritiaggarwal.pdf')
# semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')
semantic_search_resume = PDFSearchTool(pdf='./aakritiaggarwal.pdf')

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  3.97it/s]


- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [6]:
# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

## Creating Agents

In [7]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your process in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [8]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [9]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [10]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [11]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [12]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [13]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [14]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [15]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [17]:
job_application_inputs = {
    'job_posting_url': 'https://www.linkedin.com/jobs/view/3978492503',
    'github_url': 'https://github.com/aakriti1318',
    'personal_writeup': """Aakriti is an accomplished AI Engineer
    Engineering with 2 years of experience, specializing in multiple
    programming languages and frameworks. She holds a BTech Degree and a strong
    background in AI and Software Developement Role. Aakriti has successfully led
    major tech initiatives and startups, proving her ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

- Dislplay the generated `tailored_resume.md` file.

In [19]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

Aakriti Aggarwal
[Email Address] | [Phone Number] | [LinkedIn Profile] | [GitHub Profile] | [Medium Profile]

Summary:
Passionate and innovative Developer at SAP Labs with a diverse skill set in AI, software development, and writing. Proficient in Python, CSS, Jupyter Notebook, and more. Experienced in developing AI projects, contributing to various GitHub repositories, and maintaining a Digital Garden with regular blog posts.

Work Experience:
SAP Labs, Developer
- Developed and implemented AI models for various projects, focusing on model prototyping, productionalization, and enhancement.
- Contributed to projects such as GenAI, Vasitum-Document-Classifier, SelfAid, llama_parse, and aint-that-easy on GitHub.
- Collaborated with cross-functional teams to drive innovation and strategic thinking in major tech initiatives.

Skills:
- Model Prototyping
- Model Productionalization
- Model Enhancement
- Machine Learning Tools
- Proficient in Python, CSS, Jupyter Notebook
- Experience with data science tools and frameworks: Scikit, NLTK, Numpy, Pandas, TensorFlow, Keras, R, Spark
- Basic knowledge of machine learning techniques: classification, regression, clustering

Education:
- Bachelor of Science in Computer Science, [University Name]
- Master of Science in Computer Science, [University Name]
- PhD in Computer Science, [University Name]

Additional Information:
- Passionate about forming groups for Hackathons, Competitions, and Research Papers.
- Maintains a Digital Garden with blog posts on Medium covering various topics.
- Creative and engaging communication style with a desire to continually learn and share knowledge with others.
- Leadership experience in major tech initiatives, ideal for roles requiring innovation and strategic thinking.

- Dislplay the generated `interview_materials.md` file.

In [20]:
display(Markdown("./interview_materials.md"))

Interview Questions and Talking Points for Aakriti Aggarwal:

1. Can you elaborate on your experience with model prototyping, productionalization, and enhancement as mentioned in your resume?
2. How have you utilized machine learning tools such as Python, Scikit, NLTK, Numpy, Pandas, TensorFlow, Keras, R, and Spark in your previous projects?
3. Could you share a specific project where you implemented machine learning techniques like classification, regression, and clustering?
4. What motivated you to pursue a BS, MS, or PhD degree in Computer Science, and how has this academic background influenced your work in AI and software development?
5. In what ways have you demonstrated leadership in major tech initiatives, and how do you approach innovation and strategic thinking in your roles?
6. Tell us about your experience in forming groups for Hackathons, Competitions, and Research Papers. How do you see collaborative projects contributing to your growth as a developer?
7. How do you maintain your Digital Garden on Medium, and what topics do you usually cover in your blog posts?
8. Can you walk us through your contributions to GitHub projects like GenAI, Vasitum-Document-Classifier, SelfAid, llama_parse, and aint-that-easy? How do these projects showcase your expertise in AI and software development?
9. What communication style do you adopt when working on cross-functional teams, and how do you ensure effective collaboration and knowledge sharing?
10. How do you stay updated with the latest trends and advancements in AI, data science, and machine learning, and how do you incorporate this knowledge into your work at SAP Labs?

These questions and talking points aim to help Aakriti Aggarwal highlight her key skills, experiences, and achievements during the interview, showcasing her suitability for leadership roles that require innovation, strategic thinking, and a strong technical background in AI and software development.